<a href="https://colab.research.google.com/github/uio-fys-mek/md-prosjekt/blob/master/vektorisering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np

# Vektorisering

Når vi snakker om *vektorisering* i Python mener vi egentlig å utnytte funksjonaliteter og vektoroperasjoner i Python-pakken `numpy`. Numpy er en pakke som er skrevet i C/C++ og delvis Fortran - lavnivåspråk som generelt er mye raskere enn høynivåspråk som f.eks. Python. Dette gjør seg virkelig gjeldende i essensielle funksjonaliteter som for- og while-løkker. For eksempel vil en for-løkke i C/C++ gå 100 ganger (!) raskere enn en for-løkke i Python.

## Enkelt eksempel
Det som er fint med numpy-biblioteket er at vektoroperasjoner som gjøres på numpy-arrays vil kalle på funksjonaliteter skrevet i nettopp disse lavnivå språkene. På denne måten drar man fordelene av hastighetene til disse, samtidig som man beholder en ren og kort python-syntax. Dette lar oss tilsynelatende gjøre operasjoner på *hele arrays* om gangen i python-scriptene våre.

Enkelt eksempel:


In [10]:
a = np.array([1,2,3])
b = np.array([1,1,1])
c = a - b
c

array([0, 1, 2])

Dette oversettes til C/Fortran kode som tilsvarer

In [11]:
c = np.zeros_like(a)
for i in range(len(c)):
    c[i] = a[i] - b[i]
c

array([0, 1, 2])

Dette går mye raskere:

In [0]:
def dum():
    N = int(1E6)
    a = np.zeros(N)
    b = np.zeros(N)
    c = np.zeros(N)
    for i in range(N):
        a[i] = np.random.uniform()
        b[i] = np.random.uniform()
        c[i] = a[i] - b[i]

def vektorisert():
    N = int(1E6)
    a = np.random.uniform(size=N)
    b = np.random.uniform(size=N)
    c = a - b

In [13]:
%timeit dum()

1 loop, best of 3: 2.19 s per loop


In [14]:
%timeit vektorisert()

10 loops, best of 3: 31.4 ms per loop


Som vi ser, går den vektoriserte koden mye raskere. I tillegg er koden enklere og mer matematikknær.

Moralen er at man alltid bør prøve å erstatte så mange Python-løkker som mulig med vektoriserte numpy-operasjoner.

## Parvise operasjoner
I Grand Challenge må vi finne avstandene mellom hvert eneste atompar for å kunne regne ut summen av kreftene på hvert atom. I eksemplene under viser vi flere måter å sammenligne alle elementene i et array mot hverandre, både delvis og fullt vektorisert. For enkelhets skyld opererer vi på et 1D-array, men metodene kan også lett brukes på arrays med flere dimensjoner.


### Enkel tilnærming
Hvis posisjonene er lagret i `r` som har dimensjon $N \times 3 $, hvor $N$ er antallet partikler, vil koden typisk se slik ut:
```python
for i in range(N-1):
    for j in range(i+1, N):
        dr = r[j] - r[i]
```

Dette er den mest "rett frem"-måten å sammenligne array'ene på, men også den tregeste. Når antallet iterasjoner per loop ($N$) blir stort, vil bare det å loope gjennom løkkene ta en god del tid. Skal man i tillegg gjøre kompliserte beregninger inne i løkkene kan det være en fordel å ha abonnement hos Netflix. Samtidig er det veldig oversiktlig og enkelt å kun behandle ett element om gangen, hvor man har god kontroll på hva som faktisk skjer. Likevel, 


 - +Lett å forstå hva som skjer.
 - +Trenger kun ett element i minnet om gangen.
 - -Kan bli en del kode.
 - -*Ekstremt* tregt.

### Vektorisert utregning
Alle avstandsvektorene til naboene til atom $i$ kan regnes ut på én gang med
```python
for i in range(N):
    drs = r - r[i]
```
og kreftene kan regnes ut fra disse. OBS: `drs[i]` vil være $\vec{0}$.